In [197]:
import numbers
from pathlib import Path

import numpy as np
import pandas as pd
from scipy import stats

from geopy import distance

import osmnx as ox
import networkx as nx

from tqdm.autonotebook import tqdm
tqdm.pandas()

C:\Anaconda3\envs\hse\lib\site-packages\tqdm\std.py:651: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
data_path = Path('final-etarta')

In [160]:
train = pd.read_csv(data_path / 'train.csv')
valid = pd.read_csv(data_path / 'validation.csv')
test = pd.read_csv(data_path / 'test.csv')

In [161]:
train.shape

(832718, 23)

In [162]:
train.head()

,Id,main_id_locality,ETA,RTA,OrderedDate,latitude,del_latitude,longitude,del_longitude,EDA,...,GoodArrived,ready_latitude,ready_longitude,onway_latitude,onway_longitude,arrived_latitude,arrived_longitude,center_latitude,center_longitude,route
0,0,1078,226.0,188.0,2020-02-12 19:12:06,55.826019,55.825581,49.134529,49.126949,1.0,...,2020-02-12 19:18:14,55.825647,49.134115,55.826261,49.134137,55.825706,49.127136,55.794388,49.111531,gnvsIaq{jHChA??uC???OPG^F^NRzKBd@AN[r@???`@`@`...
1,1,1078,718.0,725.0,2020-02-12 19:12:22,55.795502,55.820911,49.131470,49.115360,5.0,...,2020-02-12 19:31:47,55.795591,49.132281,55.795544,49.131552,55.821531,49.115685,55.794388,49.111531,sqpsI}~zjHyAr]e@lMk@fLaBlb@i@rLKhBCdAUxEGlCg@f...
2,2,1078,612.0,764.0,2020-02-12 19:12:44,55.791050,55.819962,49.226070,49.176628,5.0,...,2020-02-12 19:28:09,55.791099,49.226066,55.791099,49.226066,55.819765,49.177432,55.794388,49.111531,auosI}mmkH?LHd@KhC??o@w@[g@m@iAUk@??{G|OiB`Ek@...
3,3,1078,1560.0,1412.0,2020-02-12 19:12:44,55.753899,55.824680,49.188519,49.093700,13.0,...,2020-02-12 19:42:41,55.754116,49.188853,55.754116,49.188853,55.824734,49.094013,55.794388,49.111531,{lhsIiffkHmKN_C?mIPwMJ??Si@gA{B??Wq@MRCJTp@hAd...
4,4,1078,1528.0,893.0,2020-02-12 19:12:45,55.822361,55.786758,49.069092,49.143501,9.0,...,2020-02-12 19:32:19,55.822483,49.069659,55.822617,49.069229,55.786936,49.143330,55.794388,49.111531,yxusI{xnjHgAfG??}IuHkAqA??pIoe@VsA??dAkG`BuH??...


Посмотрим насколько сильно ошибается текущая модель

In [163]:
train.groupby(by='main_id_locality').apply(lambda x: (x['ETA'] - x['RTA']).describe())

,count,mean,std,min,25%,50%,75%,max
main_id_locality,,,,,,,,
1078,312528.0,177.698353,222.934094,-2474.0,48.0,160.0,296.0,2119.0
22390,182536.0,80.433290,209.996653,-2200.0,-21.0,75.0,182.0,2542.0
22430,135563.0,161.315536,200.832095,-3156.0,46.0,138.0,255.0,2451.0
22438,202091.0,212.489725,256.604091,-2804.0,61.0,174.0,329.0,2396.0


In [164]:
train.groupby(by='main_id_locality').apply(lambda x: (x['EDA'] - x['RDA']).describe())

,count,mean,std,min,25%,50%,75%,max
main_id_locality,,,,,,,,
1078,312528.0,0.471468,2.637179,-43.0,0.0,0.0,1.0,43.0
22390,182536.0,0.340919,2.203042,-194.0,0.0,0.0,0.0,118.0
22430,135563.0,0.421450,2.029351,-44.0,0.0,0.0,1.0,69.0
22438,202091.0,1.114191,3.748445,-62.0,0.0,0.0,1.0,97.0


Либо выбросы, либо что-то очень странное (маршрут на 194 км короче)

Выбросы можно почистить либо глобально, либо внутри каждой группы

In [165]:
train = train.groupby(by='main_id_locality') \
    .apply(lambda x: x[(np.abs(stats.zscore(x['ETA'] - x['RTA'])) <= 3) & (np.abs(stats.zscore(x['EDA'] - x['RDA'])) <= 3)]) \
    .reset_index(drop=True)

In [166]:
train.shape

(803180, 23)

In [167]:
train_time_diff = train.groupby(by='main_id_locality').apply(lambda x: (x['ETA'] - x['RTA']).mean())
train_time_diff = pd.DataFrame(train_time_diff, columns=['time_diff']).reset_index()
train_time_diff

,main_id_locality,time_diff
0,1078,171.399381
1,22390,78.184118
2,22430,155.030171
3,22438,194.684379


In [168]:
train_dist_diff = train.groupby(by='main_id_locality').apply(lambda x: (x['EDA'] - x['RDA']).mean())
train_dist_diff = pd.DataFrame(train_dist_diff, columns=['distance_diff']).reset_index()
train_dist_diff

,main_id_locality,distance_diff
0,1078,0.111761
1,22390,0.117156
2,22430,0.235388
3,22438,0.483684


Ошибка действительно большая

Какие города в выборке

In [170]:
train['main_id_locality'].unique(), train['center_latitude'].unique(), train['center_longitude'].unique()

(array([ 1078, 22390, 22430, 22438], dtype=int64),
 array([55.7943877, 53.18333  , 51.530376 , 58.0103211]),
 array([49.1115312, 50.11667  , 45.9530257, 56.2341778]))

* 1078 - Казань/Kazan - +0 мск
* 22390 - Самара/Samara - +1 мск
* 22430 - Саратов/Saratov - +1 мск
* 22438 - Пермь/Perm - +2 мск

In [171]:
city_offset = {
    1078: 0,
    22390: 1,
    22430: 1,
    22438: 2
}

In [172]:
def time_preprocessing(df):
    df['OrderedDate'] = pd.to_datetime(df['OrderedDate'])
    df['OrderedDate'] += pd.to_timedelta(df['main_id_locality'].apply(lambda x: city_offset[x]), unit='h')
    
    df['day_of_week'] = df['OrderedDate'].dt.day_name()
    df['is_weekend'] = df['day_of_week'].isin(['Saturday', 'Sunday'])

    df['hour'] = df['OrderedDate'].dt.hour
    df['is_rush_hour'] = df['hour'].between(8, 12) | df['hour'].between(17, 19)
    df['is_night'] = df['hour'].between(22, 24) | df['hour'].between(0, 7)

    return df

In [173]:
def geo_preprocessing(df):
    df['distance'] = df.progress_apply(lambda x: distance.distance((x['latitude'], x['longitude']), (x['del_latitude'], x['del_longitude'])).km, axis=1)
    df['detour'] = df['EDA'] - df['distance']
    return df

In [175]:
def diff_preprocessing(df):
    df = df.merge(train_time_diff)
    df = df.merge(train_dist_diff)
    return df

In [179]:
preprocessings = [time_preprocessing, geo_preprocessing, diff_preprocessing]

def preprocess(df):
    for preprocessing in preprocessings:
        print(f"Start {preprocessing.__name__}")
        df = preprocessing(df)
    return df

In [180]:
train = preprocess(train)

Start time_preprocessing
Start geo_preprocessing



Start diff_preprocessing


In [181]:
train.head()

,Id,main_id_locality,ETA,RTA,OrderedDate,latitude,del_latitude,longitude,del_longitude,EDA,...,route,day_of_week,is_weekend,hour,is_rush_hour,is_night,distance,detour,time_diff,distance_diff
0,0,1078,226.0,188.0,2020-02-12 19:12:06,55.826019,55.825581,49.134529,49.126949,1.0,...,gnvsIaq{jHChA??uC???OPG^F^NRzKBd@AN[r@???`@`@`...,Wednesday,False,19,True,False,0.477554,0.522446,171.399381,0.111761
1,1,1078,718.0,725.0,2020-02-12 19:12:22,55.795502,55.820911,49.131470,49.115360,5.0,...,sqpsI}~zjHyAr]e@lMk@fLaBlb@i@rLKhBCdAUxEGlCg@f...,Wednesday,False,19,True,False,3.003986,1.996014,171.399381,0.111761
2,2,1078,612.0,764.0,2020-02-12 19:12:44,55.791050,55.819962,49.226070,49.176628,5.0,...,auosI}mmkH?LHd@KhC??o@w@[g@m@iAUk@??{G|OiB`Ek@...,Wednesday,False,19,True,False,4.469195,0.530805,171.399381,0.111761
3,3,1078,1560.0,1412.0,2020-02-12 19:12:44,55.753899,55.824680,49.188519,49.093700,13.0,...,{lhsIiffkHmKN_C?mIPwMJ??Si@gA{B??Wq@MRCJTp@hAd...,Wednesday,False,19,True,False,9.873466,3.126534,171.399381,0.111761
4,4,1078,1528.0,893.0,2020-02-12 19:12:45,55.822361,55.786758,49.069092,49.143501,9.0,...,yxusI{xnjHgAfG??}IuHkAqA??pIoe@VsA??dAkG`BuH??...,Wednesday,False,19,True,False,6.122452,2.877548,171.399381,0.111761


In [182]:
class CityInfo:    
    def __init__(self, main_id_locality : int, name: str):
        self.main_id_locality = main_id_locality 
        self.name = name
        self.place = f'{self.name}, Russia'
        
    @staticmethod
    def filter_dict(d):
        return {k: v for k, v in d.items() if isinstance(v, numbers.Number)}
    
    def add_stats(self, df, name):
        self.stats[f'{name}_speed_city'] = (df['RDA'] / df['RTA']).mean()  
        avg = df.groupby(by='hour').apply(lambda x: (x['RDA'] / x['RTA']).mean())
        avg = pd.DataFrame(avg).T.rename(columns = lambda x: f"{name}_speed_at_{x}")
        return pd.concat([self.stats, avg], axis=1)
        
    def calculate_statistics(self, df):
        G = ox.graph_from_place(self.place, network_type='drive')
        gdf = ox.gdf_from_place(self.place)
        area = ox.project_gdf(gdf).unary_union.area
        
        self.basic_stats = ox.basic_stats(G, area=area, clean_intersects=True)
        self.basic_stats  = self.filter_dict(self.basic_stats)
        self.extended_stats = ox.extended_stats(G, ecc=True, bc=True, cc=True)
        self.extended_stats  = self.filter_dict(self.extended_stats)
        
        self.stats = pd.DataFrame.from_dict({**self.basic_stats, **self.extended_stats}, orient='index').T
        self.stats['main_id_locality'] = self.main_id_locality
        
        df = df[df['main_id_locality'] == self.main_id_locality]
        
        self.stats = self.add_stats(df, 'avg')
        self.stats = self.add_stats(df[df['is_rush_hour']], 'rush_avg')
        self.stats = self.add_stats(df[df['is_night']], 'night_avg')
        self.stats = self.add_stats(df[df['is_weekend']], 'weekend_avg')
        self.stats = self.add_stats(df[~df['is_weekend']], 'work_avg')

In [183]:
cities = {
    1078: CityInfo(main_id_locality=1078, name='Kazan'),
    22390: CityInfo(main_id_locality=22390, name='Samara'),
    22430: CityInfo(main_id_locality=22430, name='Saratov'),
    22438: CityInfo(main_id_locality=22438, name='Perm')
}

In [184]:
for city in tqdm(cities.values()):
    city.calculate_statistics(train)

In [192]:
cities_info = pd.concat([city.stats for city in cities.values()])
cities_info

,n,m,k_avg,intersection_count,streets_per_node_avg,edge_length_total,edge_length_avg,street_length_total,street_length_avg,street_segments_count,...,work_avg_speed_at_14,work_avg_speed_at_15,work_avg_speed_at_16,work_avg_speed_at_17,work_avg_speed_at_18,work_avg_speed_at_19,work_avg_speed_at_20,work_avg_speed_at_21,work_avg_speed_at_22,work_avg_speed_at_23
0,7021.0,17669.0,5.033186,5604.0,2.795756,3452335.425,195.389407,1950330.999,199.053991,9798.0,...,0.007848,0.007812,0.007144,0.006290,0.006826,0.008095,0.008686,0.009017,0.009310,0.009489
0,5729.0,14738.0,5.145051,4770.0,2.874847,2694314.768,182.814138,1493328.164,182.024398,8204.0,...,0.005852,0.005836,0.005461,0.004852,0.005169,0.005990,0.006655,0.007190,0.007522,0.007771
0,5893.0,15105.0,5.126421,4575.0,2.740031,2395076.324,158.561822,1274114.934,158.118011,8058.0,...,0.006466,0.006466,0.006363,0.005756,0.006163,0.006852,0.007341,0.007745,0.007981,0.008013
0,6554.0,16438.0,5.016173,4999.0,2.747635,3344093.900,203.436787,1830806.903,203.830651,8982.0,...,0.005796,0.005851,0.005527,0.004939,0.004931,0.005800,0.006247,0.006691,0.007166,0.007474


In [193]:
def join_cities_info(df):
    return df.merge(cities_info)

In [196]:
train = join_cities_info(train)

In [198]:
train.shape

(803180, 158)

Применяем для оставшихся частей

In [199]:
valid = preprocess(valid)
valid = join_cities_info(valid)

Start time_preprocessing
Start geo_preprocessing



Start diff_preprocessing


In [200]:
test = preprocess(test)
test = join_cities_info(test)

Start time_preprocessing
Start geo_preprocessing



Start diff_preprocessing


In [205]:
preprocessed_dir = Path('preprocessed')
preprocessed_dir.mkdir(exist_ok=True)

In [206]:
train.to_csv(preprocessed_dir / 'train.csv', index=False)
valid.to_csv(preprocessed_dir / 'valid.csv', index=False)
test.to_csv(preprocessed_dir / 'test.csv', index=False)

In [210]:
def get_current_speed(x):
    if x['is_night']:
        return x[f"night_avg_speed_at_{x['hour']}"]
    
    if x['is_rush_hour']:
        return x[f"rush_avg_speed_at_{x['hour']}"]
    
    if x['is_weekend']:
        return x[f"weekend_avg_speed_at_{x['hour']}"]
    
    return x[f"work_avg_speed_at_{x['hour']}"]

In [213]:
train.progress_apply(get_current_speed, axis=1)

0         0.008250
1         0.008250
2         0.008250
3         0.008250
4         0.008250
            ...   
803175    0.006047
803176    0.006047
803177    0.006047
803178    0.006047
803179    0.006047
Length: 803180, dtype: float64

In [ ]:
train.apply(lamnda x: )

In [208]:
print(list(train.columns))

['Id', 'main_id_locality', 'ETA', 'RTA', 'OrderedDate', 'latitude', 'del_latitude', 'longitude', 'del_longitude', 'EDA', 'RDA', 'ReadyForCollection', 'ClientCollected', 'GoodArrived', 'ready_latitude', 'ready_longitude', 'onway_latitude', 'onway_longitude', 'arrived_latitude', 'arrived_longitude', 'center_latitude', 'center_longitude', 'route', 'day_of_week', 'is_weekend', 'hour', 'is_rush_hour', 'is_night', 'distance', 'detour', 'time_diff', 'distance_diff', 'n', 'm', 'k_avg', 'intersection_count', 'streets_per_node_avg', 'edge_length_total', 'edge_length_avg', 'street_length_total', 'street_length_avg', 'street_segments_count', 'node_density_km', 'intersection_density_km', 'edge_density_km', 'street_density_km', 'circuity_avg', 'self_loop_proportion', 'clean_intersection_count', 'clean_intersection_density_km', 'avg_neighbor_degree_avg', 'avg_weighted_neighbor_degree_avg', 'degree_centrality_avg', 'clustering_coefficient_avg', 'clustering_coefficient_weighted_avg', 'pagerank_max_node

In [ ]:
G = ox.graph_from_place('Yekaterinburg, Russia', network_type='drive')

In [ ]:
gdf = ox.gdf_from_place('Yekaterinburg, Russia')

area = ox.project_gdf(gdf).unary_union.area

In [ ]:
ox.plot_graph(G)